# DATS 6103 - Individual Project 2 - Piolini Andrea

## Introduction

This project is a data mining analysis that aims to make a comparison between jihadist terrorist attacks and non-jihadist terrorist attacks in Western Europe from January 1, 2011 to December 31, 2018. Jihadist attacks are defined as all the attacks perpetrated by an individual or a group inspired by the Salafist and jihadist propaganda and ideology, regardless of if they were officially affiliated with a terrorist group or they acted as “lone wolves”. The category of non-jihadist attacks comprises all the attacks perpetrated by individuals or groups motivated by any different kind of ideology, including nationalist, racial, separatist and non-Islamic religious motives. The analysis compares four main factors: the countries where the attacks took place, the modus operandi of the terrorists, their targets and number of fatalities. The Western Europe area does not include all the members of the European Union, since some of them are located in Eastern Europe, and it includes some countries that are not members of the Union, such as Switzerland and Norway. Moreover, this analysis does not take into consideration the terrorist plots disrupted by the authorities, but instead focuses exclusively on the attacks that actually took place, whether successful or not. 

This research considers the data starting from 2011 because it is considered a transition year for jihadism in Western Europe, as well as jihadism in general. Indeed, in 2011 there was widespread optimism among European security agencies due to the perception that jihadism had become a spent force, which was no longer able to perpetrate deadly attacks in Western Europe. However, this perception was completely shattered when the Arab Spring deteriorated into a jihadist front and the civil war in Syria became an unprecedented magnet for jihadist groups and foreign fighters. The Syrian conflict became a game changer for jihadism and the birth of Islamic State of Iraq and Syria (ISIS) within the war increased its international echo. 

The data this project is based on was retrieved from the Global Terrorism Database, an open-source database maintained by the National Consortium for the Study of Terrorism and Responses to Terrorism at the University of Maryland, College Park, in the United States. As of October 2019, the database includes information on over 190,000 terrorist events occurred around the world from 1970 to 2018. 

### Project's Goal and Structure

The project seek to make a comparison between jihadist and non-jihadist terrorist attacks in Western Europe to find out if there are patterns and/or associations related to the countries where the attacks took place, the modus operandi, the targets and the number of fatalities caused by the attacks. 

This project is divided in the following sections:

-	Data Cleaning Process and Preliminary Analysis 
-	Part 1: Comparison by Country
-	Part 2: Comparison by Modus Operandi
-	Part 3: Comparison by Types of Targets
-	Part 4: Comparison by Fatalities
-	Maps 
-	Conclusions

## Data Cleaning Process and Preliminary Analysis

In [ ]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#loading the data
GTD = pd.read_excel('GTD1.xlsx')

In [ ]:
GTD.head()

In [ ]:
#looking at datatypes
GTD.dtypes

In [ ]:
#getting the shape of the data
GTD.shape

In [ ]:
#cleaning the data
GTD.rename(columns ={'iyear' : 'Year', 'country_txt' : 'Country', 'region_txt' : 'Region', 'city' : 'City', 'latitude' : 'Latitude', 'longitude' : 'Longitude', 'attacktype1_txt' : 'Attack Type', 'attacktype2_txt' : 'Attack Type 2', 'targtype1_txt' : 'Target Type', 'targtype2_txt' : 'Target Type 2', 'gname' : 'Group Name', 'weaptype1_txt' : 'Weapon Type', 'nkill' : 'Number of People Killed', 'nwound' : 'Number of People Wounded'}, inplace = True)
GTD = GTD.ix[:, ['Year', 'Country', 'Region', 'City', 'Latitude', 'Longitude', 'Attack Type', 'Attack Type 2', 'Target Type', 'Target Type 2', 'Group Name', 'Weapon Type', 'Number of People Killed', 'Number of People Wounded']]
GTD.head()

In [ ]:
GTD.tail()

In [ ]:
#Getting the countries in Western Europe for the years between 2011 and 2018, included
GTD = GTD[(GTD.Year >= 2011) & (GTD.Year <= 2018) & (GTD.Region == 'Western Europe')]
GTD.head()

In [ ]:
GTD.tail()

In [ ]:
GTD.shape

In [ ]:
#creating a new data frame GTD_no_names to compare the number of attacks perpetrated by unknown and known groups
GTD_no_names = GTD.copy()
GTD_no_names.loc[GTD['Group Name'] != 'Unknown', GTD_no_names.columns[10]] = 'Known Group'
GTD_no_names.head(10)

In [ ]:
GTD_no_names.tail()

In [ ]:
#creating a list to plot the number of attacks by known and unknow groups
totalnumberattacks = len(GTD_no_names)
numberunknownattacks = len(GTD_no_names[GTD_no_names['Group Name'] == 'Unknown'])
numberknownattacks = len(GTD_no_names[GTD_no_names['Group Name'] != 'Unknown'])
numberunknownattacks
totalnumberattacks = [numberunknownattacks, numberknownattacks]
totalnumberattacks

In [ ]:
#plotting unknown and know attacks in percentages
plt.figure(figsize=(7,7))

labels = ['Unknown Group', 'Known Group']
colors = ['Chocolate', 'Khaki']
plt.legend()
plt.pie(totalnumberattacks, colors = colors, labels=labels, shadow = True, startangle = 140, autopct='%1.1f%%')
plt.title('Number of Terrorist Attacks Perpetrated by Known And Unknown Groups', fontsize=16)
plt.show()

In [ ]:
#removing unknown perpetrators
GTD = GTD[GTD['Group Name'] != 'Unknown']
GTD

In [ ]:
GTD.shape

In [ ]:
#looking at the terrorist groups
GTD['Group Name'].count

In [ ]:
#displaying all the terrorist groups 
pd.set_option('display.max_rows', 200)
GroupNames = GTD.groupby('Group Name').size()
GroupNames

In [ ]:
#making another copy of GTD and including all the islamist groups in a macro group called Islamist/Jihadist
GTD_Jihad_non_Jihad = GTD.copy()
Jihadist_Groups = ['Al-Qaida in the Arabian Peninsula (AQAP)', 'Islamic State of Iraq and the Levant (ISIL)', 'Jihadi-inspired extremists', 'Muslim extremists', 'Sunni Muslim extremists']

#looping in the jihadist groups to change their name to Islamist/Jihadist
for group in Jihadist_Groups:
    GTD_Jihad_non_Jihad.loc[GTD['Group Name'] == group, GTD_no_names.columns[10]] = 'Islamist/Jihadist'   
GTD_Jihad_non_Jihad.groupby('Group Name').size()

In [ ]:
#creating another macro group called Non-Jihadist
GTD_Jihad_non_Jihad.loc[GTD_Jihad_non_Jihad['Group Name'] != 'Islamist/Jihadist', GTD_no_names.columns[10]] = 'Non-Jihadist'
GTD_Jihad_non_Jihad.groupby('Group Name').size()

In [ ]:
#creating a list to plot the number of attacks by jihadist and non-jihadist groups
numberjihadistattacks = len(GTD_Jihad_non_Jihad[GTD_Jihad_non_Jihad['Group Name'] == 'Islamist/Jihadist'])
numberotherattacks = len(GTD_Jihad_non_Jihad[GTD_Jihad_non_Jihad['Group Name'] == 'Non-Jihadist'])
totalnumberknownattacks = [numberjihadistattacks, numberotherattacks]
totalnumberknownattacks

In [ ]:
#plotting the number of attacks by known and unknow groups

plt.figure(figsize=(7,7))

labels = ['Islamist/Jihadist', 'Non-Jihadist']
colors = ['Forestgreen', 'Silver']
plt.legend()
plt.pie(totalnumberknownattacks, colors = colors, labels=labels, shadow = True, startangle = 140, autopct='%1.1f%%')
plt.title('Number of Terrorist Attacks Perpetrated by Jihadist and Non-Jihadist Groups', fontsize=16)
plt.show()

In [ ]:
#creating a new column Number of Terrorist Attacks to calculate the total number of attack in GTD_Jihad_non_Jihad
GTD_Jihad_non_Jihad['Number of Terrorist Attacks'] = 1

In [ ]:
totnumberattacks = GTD_Jihad_non_Jihad['Number of Terrorist Attacks'].sum()
totnumberattacks

In [ ]:
#changing the Year format from integer to datetime
pd.to_datetime(GTD_Jihad_non_Jihad.Year, format = '%Y')

In [ ]:
#creating a frequency table that displays the number of jihadist and non-jihadist attacks per year
ByYear = pd.crosstab(GTD_Jihad_non_Jihad['Year'], GTD_Jihad_non_Jihad['Group Name'])
ByYear = pd.DataFrame(ByYear)
ByYear

In [ ]:
#plotting the numbers of jihadist and non-jihadist terrorist attacks over the years 
colors = ['#228B22', '#C0C0C0']
ax = ByYear.plot(colors = colors, figsize= (20,10), fontsize = 13, linewidth = 3.0)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Year', fontsize = 13)
ax.set_ylabel('Number of Attacks', fontsize = 13)
plt.title('Number of Terrorist Attacks Perpetrated by Jihadist and Non-Jihadist Groups Over Time', fontsize = 16)
plt.show()

## Comparison by Country

From this section on, I just consider the attacks perpetrated by known terrorist groups

In [ ]:
#creating a frequency table that displays the number of jihadist and non-jihadist attacks per country
ByCountry = pd.crosstab(GTD_Jihad_non_Jihad['Country'], GTD_Jihad_non_Jihad['Group Name'])
ByCountry = pd.DataFrame(ByCountry)
ByCountry

In [ ]:
#plotting the numbers of jihadist and non-jihadist terrorist attacks by country 
colors = ['#228B22', '#C0C0C0']
ax = ByCountry.plot(kind = 'barh', color = colors, figsize= (20,10), fontsize = 13)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Number of Attacks', fontsize = 13)
ax.set_ylabel('Country', fontsize = 13)
plt.title('Number of Terrorist Attacks Perpetrated by Jihadist and Non-Jihadist Groups by Country', fontsize = 16)
plt.show()

In [ ]:
#plotting the numbers of jihadist terrorist attacks by country
#creating a new dataframe df1 without the non-jihadist attacks first
df1 = ByCountry.drop('Non-Jihadist', axis = 1)
df1 = df1.sort_values(by = 'Islamist/Jihadist')
df1 = df1[(df1.T != 0).any()] #eliminating the 0s so they won't appear in the graph later
df1

In [ ]:
#plotting df1
colors = ['#228B22']

ax = df1.plot(kind = 'barh', color = colors, figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Attacks', fontsize = 13)
ax.set_ylabel('Country', fontsize = 13)
plt.title('Number of Terrorist Attacks Perpetrated by Jihadist Groups by Country', fontsize = 16)
plt.show()

In [ ]:
#plotting the numbers of non-jihadist terroist attacks by country 
#creating a new dataframe df2 without the jihadist attacks
df2 = ByCountry.drop('Islamist/Jihadist', axis = 1)
df2 = df2.sort_values(by = 'Non-Jihadist')
df2 = df2[(df2.T != 0).any()] #eliminating the 0s so they won't appear in the graph later
df2

In [ ]:
#plotting df2
colors = ['#C0C0C0']

ax = df2.plot(kind = 'barh', color = colors, figsize= (20,10),  fontsize = 13, legend = None)
ax.set_xlabel('Number of Attacks', fontsize = 13)
ax.set_ylabel('Country', fontsize = 13)
plt.title('Number of Terrorist Attacks Perpetrated by Non-Jihadist Groups by Country', fontsize = 16)
plt.show()

## Comparison by Modus Operandi

In [ ]:
#displaying all the modus operandi 
pd.set_option('display.max_rows', 200)
AttackTypes = GTD.groupby('Attack Type').size()
AttackTypes

In [ ]:
#displaying all the secondary types of attacks from the column Attacks Type 2
pd.set_option('display.max_rows', 200)
AttackTypes = GTD.groupby('Attack Type 2').size()
AttackTypes

In [ ]:
#converting NaN to a string 'None'
GTD_Jihad_non_Jihad = GTD_Jihad_non_Jihad.fillna('None')
GTD_Jihad_non_Jihad

In [ ]:
#changing the name of the values in the column Attack Type 1 to Mixed Type for the attacks that also have a secondary type of attack
GTD_Jihad_non_Jihad.loc[GTD_Jihad_non_Jihad['Attack Type 2'] != 'None', GTD_no_names.columns[6]] = 'Mixed Type'
AttackTypes = GTD_Jihad_non_Jihad.groupby('Attack Type').size()
AttackTypes

In [ ]:
#creating a frequency table that displays the modus operandi used by jihadist and non-jihadist groups
ByAttackType = pd.crosstab(GTD_Jihad_non_Jihad['Attack Type'], GTD_Jihad_non_Jihad['Group Name'])
ByAttackType = pd.DataFrame(ByAttackType)
ByAttackType

In [ ]:
#plotting the types of attack 
colors = ['#228B22', '#C0C0C0']
ax = ByAttackType.plot(kind = 'bar', color = colors, figsize= (20,10), fontsize = 13)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Attack Type', fontsize = 13)
ax.set_ylabel('Number of Attacks', fontsize = 13)
plt.title('Number of Terrorist Attacks Perpetrated by Jihadist and Non-Jihadist Groups by Type of Attack', fontsize = 16)
plt.show()

In [ ]:
#plotting the numbers of jihadist terrorist attacks by country
#creating a new dataframe df3 without the non-jihadist attacks first
df3 = ByAttackType.drop('Non-Jihadist', axis = 1)
df3 = df3[(df3.T != 0).any()] #eliminating the 0s so they won't appear in the pie chart later
df3

In [ ]:
#plotting the numbers of non-jihadist terroist attacks by country 
#creating a new dataframe df4 without the jihadist attacks
df4 = ByAttackType.drop('Islamist/Jihadist', axis = 1)
df4 = df4[(df4.T != 0).any()] #eliminating the 0s so they won't appear in the pie chart later
df4

In [ ]:
#plotting df3 and df4 using a pie chart to see percentages
colors1 = ['Red', 'Orange', 'Darkkhaki', 'Lightseagreen', 'Mediumvioletred', 'Palevioletred', 'Grey', 'Gold']
df3.plot(kind='pie', colors = colors1, figsize = (12,12), shadow = True, startangle = 140, 
         autopct='%1.1f%%', subplots=True, labels=None, wedgeprops={'linewidth' : 2, 'edgecolor' : "white"}, 
         explode=(0.1,0,0,0,0,0,0,0))
labels1 = ['Armed Assault', 'Bombing/Explosion', 'Facility/Infrastructure Attack', 'Hijacking', 'Hostage Taking (Barricade Incident)', 'Hostage Taking (Kidnapping)', 'Mixed Type', ' Unarmed Assault']
plt.title('Modus Operandi Used by Jihadist Groups', fontsize=16)
plt.ylabel('')
plt.legend(labels1)
plt.show()

plt.figure(figsize=(10,10))

colors2 = ['Red','Darkgreen', 'Orange', 'Darkkhaki', 'Lightseagreen', 'Mediumvioletred', 'Grey', 'Gold','Yellowgreen']
df4.plot(kind='pie', colors = colors2, figsize = (14,14), startangle = 140, autopct='%1.1f%%', shadow= True, 
         subplots=True, labels=None,  wedgeprops={'linewidth' : 2, 'edgecolor' : "white"}, explode=(0,0,0.1,0.1,0,0,0,0,0))
labels2 = ['Armed Assault', 'Assassination', 'Bombing/Explosion', 'Facility/Infrastructure Attack', 'Hijacking', 'Hostage Taking (Kidnapping)', 'Mixed Type', ' Unarmed Assault', 'Unknown']
plt.title('Modus Operandi Used by Non-Jihadist Groups', fontsize=16)
plt.ylabel('')
plt.legend(labels2)
plt.show() 

## Comparison by Target

In [ ]:
#changing the name of the values in the column Target Type 1 to Multiple Targets for the attacks that had different types of targets 
GTD_Jihad_non_Jihad.loc[GTD_Jihad_non_Jihad['Target Type 2'] != 'None', GTD_no_names.columns[8]] = 'Multiple Targets'
TargetTypes = GTD_Jihad_non_Jihad.groupby('Target Type').size()
TargetTypes

In [ ]:
GTD_Jihad_non_Jihad

In [ ]:
#creating a frequency table that displays the diefferent targets of the attacks
ByTarget = pd.crosstab(GTD_Jihad_non_Jihad['Target Type'], GTD_Jihad_non_Jihad['Group Name'])
ByTarget = pd.DataFrame(ByTarget)
ByTarget

In [ ]:
#plotting the target types 
colors = ['#228B22', '#C0C0C0']

ax = ByTarget.plot(kind = 'bar', color = colors, figsize= (20,10), fontsize = 13)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Target Type', fontsize = 13)
ax.set_ylabel('Number of Attacks', fontsize = 13)
plt.title('Target Types Hit by Jihadist and Non-Jihadist Attacks', fontsize= 16)
plt.show()

In [ ]:
#plotting the numbers of jihadist terrorist attacks by country
#creating a new dataframe df5 without the non-jihadist attacks first
df5 = ByTarget.drop('Non-Jihadist', axis = 1)
df5 = df5[(df5.T != 0).any()]
df5 = df5.sort_values(by = 'Islamist/Jihadist')
df5

In [ ]:
#plotting the numbers of non-jihadist terroist attacks by country 
#creating a new dataframe df6 without the jihadist attacks
df6 = ByTarget.drop('Islamist/Jihadist', axis = 1)
df6 = df6[(df6.T != 0).any()] 
df6 = df6.sort_values(by = 'Non-Jihadist')
df6

In [ ]:
#plotting df5 and df6
colors = ['#228B22']

df5.plot(kind = 'barh', color = colors, figsize = (20,10),  fontsize = 13, legend = None)
plt.xlabel('Number of Attacks',  fontsize = 13)
plt.ylabel('Target Type',  fontsize = 13)
plt.title('Types of Targets Hit by Jihadist Terrorist Attacks',  fontsize = 16)
plt.show()

color = ['#C0C0C0']

df6.plot(kind = 'barh', color = color, figsize = (20,10),  fontsize = 13, legend = None)
plt.xlabel('Number of Attacks',  fontsize = 13)
plt.ylabel('Target Type',  fontsize = 13)
plt.title('Types of Targets Hit by Non-Jihadist Terrorist Attacks',  fontsize = 16)
plt.show()

## Comparison by Killings

In [ ]:
#converting the Number of People Killed column from float to integer
GTD_Jihad_non_Jihad['Number of People Killed'] = GTD_Jihad_non_Jihad['Number of People Killed'].astype(int)
GTD_Jihad_non_Jihad

In [ ]:
#summing the number of fatalities for Jihadist and Non-Jihadist attacks
ByKilling = GTD_Jihad_non_Jihad.groupby('Group Name').sum()
ByKilling = ByKilling.drop('Year', axis = 1) #dropping the unnecessary columns
ByKilling = ByKilling.drop('Latitude', axis = 1)
ByKilling = ByKilling.drop('Longitude', axis = 1)
ByKilling

In [ ]:
#creating a new column called Ratio displaying the average number of fatalities per single terrorist attack
ByKilling['Ratio'] = ByKilling['Number of People Killed'] / ByKilling['Number of Terrorist Attacks']
ByKilling

In [ ]:
#plotting the data frame ByKilling in a pie chart to compare the number of fatalities caused by the two groups in percentages
ByKilling = ByKilling.drop(['Number of Terrorist Attacks', 'Ratio'], axis = 1)

colors = ['Forestgreen', 'Silver']
ByKilling.plot(kind = 'pie', colors = colors, figsize = (7,7), subplots = True, 
               legend = False, shadow = True, startangle = 140, autopct='%1.1f%%')
labels = ['Islamist/Jihadist', 'Non-Jihadist']
plt.ylabel('')
plt.title('Fatalities Caused by Jihadist and Non-Jihadist Attacks', fontsize= 15)
plt.show()

In [ ]:
#creating a new data frame ByKillingYear to compare the number of terrorist attacks and the number of people killed per year
ByKillingYear = GTD_Jihad_non_Jihad.groupby('Year').sum()
ByKillingYear

In [ ]:
#adding the number of attacks perpetrated by the two groups per year
ByKillingYear = pd.concat([ByKillingYear, ByYear], axis = 1)
ByKillingYear

In [ ]:
#dropping the columns Latitude and Longitude because they are not needed
ByKillingYear = ByKillingYear.drop('Latitude', axis = 1)
ByKillingYear = ByKillingYear.drop('Longitude', axis = 1)
ByKillingYear

In [ ]:
#plotting ByKillingYear
colors = ['Black', 'Orange', 'Forestgreen', 'Silver']
ax = ByKillingYear.plot(color = colors, figsize= (20,10), fontsize = 13, linewidth = 3.0)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Year', fontsize = 13)
ax.set_ylabel('Number', fontsize = 13)
plt.title('Number of Terrorist Attacks and Total Number People Killed Over Time', fontsize= 16)
plt.show()

In [ ]:
#creating a new data frame OnlyJihad containing only the attacks perpetrated by Islamist groups
#grouping OnlyJihad by year and getting rid of the unnecessary columns
OnlyJihad = GTD_Jihad_non_Jihad.loc[GTD_Jihad_non_Jihad['Group Name'] == 'Islamist/Jihadist']
JihadKillingsYears = OnlyJihad.groupby('Year').sum()
JihadKillingsYears = JihadKillingsYears.drop(['Latitude', 'Longitude', 'Number of Terrorist Attacks'], axis = 1)

#creating a new data frame OnlyNonJihad containing only the attacks perpetrated by Islamist groups
OnlyNonJihad = GTD_Jihad_non_Jihad.loc[GTD_Jihad_non_Jihad['Group Name'] == 'Non-Jihadist']

#grouping OnlyNonJihad by year and getting rid of the unnecessary columns
NonJihadKillingsYears = OnlyNonJihad.groupby('Year').sum()
NonJihadKillingsYears = NonJihadKillingsYears.drop(['Latitude', 'Longitude', 'Number of Terrorist Attacks'], axis = 1)

#plotting JihadKillingsYears and NonJihadKillingsYears in the same graph to compare them 
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(JihadKillingsYears, color = '#228B22', linewidth = 3.0)
ax.plot(NonJihadKillingsYears, color = '#C0C0C0', linewidth = 3.0)
ax.set_xlabel('Year')
ax.set_ylabel('Number of Fatalities')
plt.title('Fatalities Caused by Jihadist and Non-Jihadist Attacks Over Time', fontsize= 15)
plt.show()

## Maps

In [ ]:
#signing in with Plotly token
py.sign_in('AndrePiolini', '7WymlJ4rlRgAoVzL8KNI')

In [ ]:
#checking the Plotly version I am using 
print(plotly.__version__)

In [ ]:
#plotting every single Jihadist attack on a map of Western Europe. Every dot indicates the city where the attack took place, 
#the type of attack, the target and the number of people killed (if any) in the attack
#in this case, Jihadist terrorist attacks will be displayed in black
fig = px.scatter_mapbox(OnlyJihad, lat="Latitude", lon="Longitude", hover_name="City", hover_data=["Attack Type", "Target Type", "Number of People Killed"],
                        color_discrete_sequence=["black"], zoom=3, height=500)
#setting the type of map
fig.update_layout(mapbox_style="open-street-map")
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#plotting every single Non-Jihadist attack on a map of Western Europe. Every dot indicates the city where the attack took place, 
#the type of attack, the target and the number of people killed (if any) in the attack
#in this case, Non-Jihadist terrorist attacks will be displayed in red
fig = px.scatter_mapbox(OnlyNonJihad, lat="Latitude", lon="Longitude", hover_name="City", hover_data=["Attack Type", "Target Type", "Number of People Killed"],
                        color_discrete_sequence=["red"], zoom=3, height=500)
#setting the type of map
fig.update_layout(mapbox_style="open-street-map")
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#plotting a density map displaying the number of Jihadist terrorist attacks
fig = go.Figure(go.Densitymapbox(lat=OnlyJihad.Latitude, lon=OnlyJihad.Longitude, z=OnlyJihad['Number of Terrorist Attacks'], 
                                 radius=10, zmin = 0, zmax = 3))
#setting the type of map, the center and the zoom level
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=7.28138, mapbox_center_lat = 47.33571, mapbox_zoom=3)
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#plotting a density map displaying the number of Non-Jihadist terrorist attacks
fig = go.Figure(go.Densitymapbox(lat=OnlyNonJihad.Latitude, lon=OnlyNonJihad.Longitude, 
                                 z=OnlyNonJihad['Number of Terrorist Attacks'], radius=10, zmin = 0, zmax = 5))
#setting the type of map, the center and the zoom level
fig.update_layout(mapbox_style="open-street-map", height = 500, mapbox_center_lon=7.28138, mapbox_center_lat = 47.33571, 
                  mapbox_zoom=3)
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Conclusions

- Countries with the highest number of second-generation Muslim experienced more Jihadist terrorism than others. It will likely be this way in the future as well although there are other important factors that might influence the rise of Islamist extremism.
- Divided countries and/or countries with serious socioeconomic problems experienced more Jihadist terrorism than others. It will most likely remain this way in the future as well.
- Armed assault is the preferred modus operandi of Jihadist terrorists, as their goal is usually to carry out mass casualties. 
- Bombing/Explosions and facility and infrastructure attacks are the preferred modus operandi of Non-Jihadist terrorists. Non-jihadist attackers tend to carry out more symbolic attacks, to deliver a message or to threaten people and institutions. Facility and infrastructure attacks such as incendiary attacks and vandalism are very common among anarchist and political extremists to warn, intimidate and scare.
- Jihadist attacks are overall more complex and well-articulated than Non-jihadist attacks, as indicated by the fact that Jihadists have often carried out attacks combining different types of attacks and hitting multiple targets.
- Private citizens are the preferred target for both groups as they are the easiest targets to hit since for the state it is realistically impossible to protect every single person. It will always be this way.
- Police and the military are common targets for both groups. They represent the armed branch of the state all terrorist fight against. 
- Jihadists have no interest in targeting governmental targets, whereas Non-jihadists target the government frequently with symbolic attacks.
- Jihadist attacks are high-lethality strikes, whose goal is killing indiscriminately the largest number of people possible. Non-Jihadists usually carry out symbolic strikes, and their goal is to dramatize and call the attention to their ideals.
- The frequency of attacks and the number of deaths of both Jihadist and Non-Jihadist attacks have been declining. This is due to an improved effort of intelligence and law enforcement agencies and some external facts, such as the partial defeat of ISIS in Syria. In the short term, both the frequency of attacks and the number of deaths will likely keep following this trend. 


